In [ ]:
# Install all required packages
!pip install streamlit google-generativeai pyngrok

In [ ]:
import os
import zipfile

# Define file paths
zip_file_path = "/content/fashion_data.zip"
extract_dir = "/content/fashion_data_extracted"

# Unzip the file
if os.path.exists(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Successfully unzipped '{zip_file_path}' to '{extract_dir}'")
else:
    print(f"Error: '{zip_file_path}' not found. Please upload your zip file.")

Successfully unzipped '/content/fashion_data.zip' to '/content/fashion_data_extracted'


In [16]:
%%writefile app.py

import streamlit as st
import google.generativeai as genai
import os
import glob
import datetime

# --- 1. Load Knowledge Base (Your Fashion Data) ---
@st.cache_data
def load_knowledge_base():
    """
    Loads all .txt files from the extracted data directory into a single string.
    """
    knowledge = ""
    file_paths = glob.glob("/content/fashion_data_extracted/fashion_data/*.txt")

    if not file_paths:
        st.error("Error: No fashion data files found in '/content/fashion_data_extracted/fashion_data/'.")
        return None

    for f_path in file_paths:
        try:
            with open(f_path, 'r', encoding='utf-8') as f:
                knowledge += f.read() + "\n\n"
        except Exception as e:
            st.error(f"Error reading {f_path}: {e}")
    return knowledge

# --- 2. Configure Gemini API ---
try:
    GEMINI_API_KEY = os.environ.get('GOOGLE_API_KEY')
    if not GEMINI_API_KEY:
        st.error("GOOGLE_API_KEY not found in environment. Please set it in Colab Secrets (🔑).")
        model = None
    else:
        genai.configure(api_key=GEMINI_API_KEY)
        # Initialize the chat model
        # --- THIS LINE IS CORRECTED ---
        model = genai.GenerativeModel('models/gemini-flash-latest')
except Exception as e:
    st.error(f"Error configuring Gemini: {e}")
    model = None

# --- 3. Gemini Query Function (RAG + Chat History) ---
def get_fashion_recommendation(knowledge_base, chat_history, user_query):
    """
    Generates a fashion recommendation using Gemini, based on the provided
    knowledge base and chat history.
    """
    if not model:
        return "Gemini model is not initialized. Please check your API key."

    if not knowledge_base:
        return "Knowledge base is empty. Cannot provide recommendations."

    # This is the "Retrieval-Augmented Generation" (RAG) prompt.
    system_prompt = f"""
    You are a friendly and expert fashion outfit recommendation assistant.
    Your task is to answer the user's fashion request based on the chat history and your KNOWLEDGE BASE.

    **RULES:**
    1.  Base your answer *only* on the information provided in the "KNOWLEDGE BASE" below.
    2.  Use the "CHAT HISTORY" to understand the user's previous questions and context (like follow-up questions).
    3.  If no exact match is found in the KNOWLEDGE BASE, find the *closest* match and adapt it.
    4.  **IMAGE LINK:** After your text recommendation, you MUST provide a clickable markdown link to a Google Image search for the main outfit.
        Example: `[See examples of this outfit](https://www.google.com/search?tbm=isch&q=Red+Anarkali+Suit+Zari+Work)`
    5.  If the query is unrelated to fashion (like "what is the capital of France?"), use the "Out-of-Context Queries" examples to politely decline.

    ---
    KNOWLEDGE BASE:
    {knowledge_base}
    ---

    CHAT HISTORY:
    {chat_history}
    ---

    NEW USER REQUEST:
    "{user_query}"
    """

    try:
        # Pass the full prompt to generate_content
        response = model.generate_content(system_prompt)
        return response.text
    except Exception as e:
        return f"An error occurred while contacting the Gemini API: {e}"

# --- 4. Feedback Logger ---
def log_feedback(feedback):
    """Appends user feedback to a log file."""
    try:
        with open("/content/user_feedback.log", "a", encoding="utf-8") as f:
            f.write(f"--- Feedback at {datetime.datetime.now()} ---\n")
            f.write(f"{feedback}\n\n")
        return True
    except Exception as e:
        st.error(f"Error logging feedback: {e}")
        return False

# --- 5. Streamlit UI (The Website) ---
st.set_page_config(page_title="AI Fashion Advisor", layout="wide")
st.title("👗 AI Fashion Advisor 👠")
st.write("Welcome! I'm your personal fashion bot. Ask me for outfit advice!")

# Load the fashion data
knowledge_base = load_knowledge_base()

# Initialize chat history in session state
if "messages" not in st.session_state:
    st.session_state.messages = []
    st.session_state.messages.append({
        "role": "assistant",
        "content": "What's the occasion? (e.g., 'I'm a 30-year-old man going to a wedding in Hyderabad')"
    })

# --- Main App Layout ---
col1, col2 = st.columns([2, 1])

with col1:
    st.subheader("Chat")

    # Display chat messages
    chat_container = st.container(height=500)
    with chat_container:
        for message in st.session_state.messages:
            with st.chat_message(message["role"]):
                st.markdown(message["content"])

    # Chat input
    if prompt := st.chat_input("What should I wear?"):
        # Add user message to history and display it
        st.session_state.messages.append({"role": "user", "content": prompt})
        with chat_container:
            with st.chat_message("user"):
                st.markdown(prompt)

        # Generate and display bot response
        with chat_container:
            with st.chat_message("assistant"):
                with st.spinner("Finding the perfect outfit..."):
                    if knowledge_base and model:
                        # Create a simple history string for the prompt
                        history_for_prompt = "\n".join(
                            [f"{m['role']}: {m['content']}" for m in st.session_state.messages]
                        )

                        response = get_fashion_recommendation(knowledge_base, history_for_prompt, prompt)
                        st.markdown(response)
                        # Add bot response to history
                        st.session_state.messages.append({"role": "assistant", "content": response})
                    else:
                        st.error("Knowledge base or Gemini model is not loaded.")

with col2:
    st.subheader("Feedback Pipeline")
    st.write("Help the bot learn! If a recommendation was wrong or could be better, please leave your feedback.")

    feedback_text = st.text_area("Your feedback:", height=150, placeholder="e.g., 'For a Goa beach, you should also recommend Kaftans for women.'")

    if st.button("Submit Feedback"):
        if feedback_text:
            if log_feedback(feedback_text):
                st.success("Thank you! Your feedback has been logged.")
                st.balloons()
            else:
                st.error("Failed to log feedback.")
        else:
            st.warning("Please enter some feedback first.")

Overwriting app.py


In [17]:
# Run this cell ONCE to create the empty log file
!touch /content/user_feedback.log
print("Created /content/user_feedback.log")

Created /content/user_feedback.log


In [18]:
from pyngrok import ngrok, conf
import google.colab.userdata
import getpass
import os

# Kill any existing Streamlit or ngrok processes
!killall streamlit
!killall ngrok

# --- 1. Get NGROK Token ---
NGROK_AUTH_TOKEN = google.colab.userdata.get('NGROK_AUTHTOKEN')
if not NGROK_AUTH_TOKEN:
    print("NGROK_AUTHTOKEN not found in secrets. Please add it.")
    NGROK_AUTH_TOKEN = getpass.getpass("Enter your ngrok authtoken: ")

# --- 2. Get GOOGLE API Key ---
GOOGLE_KEY = google.colab.userdata.get('GOOGLE_API_KEY')
if not GOOGLE_KEY:
    print("GOOGLE_API_KEY not found in secrets. Please add it.")
else:
    print("Successfully retrieved GOOGLE_API_KEY.")

# --- 3. Run Streamlit with the API key ---
os.environ['GOOGLE_API_KEY'] = GOOGLE_KEY
conf.get_default().auth_token = NGROK_AUTH_TOKEN

!streamlit run app.py &> /dev/null &

# --- 4. Configure and connect ngrok ---
try:
    import time
    time.sleep(5)

    public_url = ngrok.connect(8501)
    print("✨ Your Fashion Website is LIVE! ✨")
    print(f"Click here to open: {public_url}")

except Exception as e:
    print(f"Error starting ngrok: {e}")
    print("Please ensure your NGROK_AUTHTOKEN secret is set correctly and try again.")

Successfully retrieved GOOGLE_API_KEY.
✨ Your Fashion Website is LIVE! ✨
Click here to open: NgrokTunnel: "https://gonadial-deprecatively-jamila.ngrok-free.dev" -> "http://localhost:8501"


Killed any running streamlit and ngrok processes.


Attempting to kill old processes...
streamlit: no process found
ngrok: no process found
Kill commands executed. Should be clear now.
